Hi everyone, hope everyone is doing well. This is my first kaggle kernel. I tried my best to bring out the insights in data. Hope you like it.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/vehicle-dataset-from-cardekho'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

**Having an overlook to data**

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.shape

**No Null Values**

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

**Wondering why a vehicle has 3 owners**

In [ ]:
df['Owner'].unique()

In [ ]:
df['Year'].unique()

**Present Price seem to have some outliers. We'll look it later**

In [ ]:
sns.boxplot(df['Present_Price'])

> We can conclude a lot from this boxplot. 

**1. Diesel vehicles are highly priced and sold. The 3rd Quartile is much higher than median, it shows that some vehicles are highly expensive
2. Petrol vehicle are cheeper than diesel and cng vehicles
3. There are not lot of cng vehicles bought.**

In [ ]:
sns.boxplot('Fuel_Type', 'Present_Price', data = df)
plt.grid(axis = 'both')

**Catplot shows frequency of Automated and Manual cars**

In [ ]:
plt.figure(figsize = (15,15))
sns.catplot(y = 'Transmission', hue = 'Year',kind = 'count', data = df)
plt.grid(axis = 'x')

**Some of the expensive cars**

In [ ]:
df[df['Present_Price'] > 24].value_counts()

***The Dataset is about car price prediction but we got some two wheelers down. All the blue points down are of two wheelers which we will remove later since we cannot include 2 wheelers to predict price of 4 wheelers.***

In [ ]:
plt.figure(figsize = (16,3))
plt.scatter(df['Car_Name'], df['Present_Price']>15)
plt.xticks(rotation = 90);

***All the low bars are of two wheelers which if included in the dataset for prediction will make prediction very inaccurate.***

In [ ]:
plt.figure(figsize = (16,6))
sns.barplot(df['Car_Name'], df['Present_Price'])
plt.xticks(rotation = 90);

In [ ]:
plt.figure(figsize = (16,6))
sns.barplot(df['Car_Name'], df['Selling_Price'])
plt.xticks(rotation = 90);

****All 2 wheelers 'Present_Price' was below 2.28 lacs. So removed all.****

In [ ]:
df1 = df[~(df['Present_Price'] < 2.28)]
df1.shape

**Finding correlation**

In [ ]:
df.corr()

****Since 'Owner' made no significant impact to our dataset. We will drop the column****

In [ ]:
df1.drop(['Car_Name','Owner'], axis = 1, inplace = True)

* **FEATURE ENGINEERING**> 

In [ ]:
df1['Fuel_Type'].unique()

In [ ]:
df1['Seller_Type'].unique()

In [ ]:
df1['Transmission'].unique()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [3])], remainder = 'passthrough')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['Seller_Type'] = le.fit_transform(df1['Seller_Type'])

In [ ]:
df1['Transmission'] = le.fit_transform(df1['Transmission'])

In [ ]:
y = df1['Selling_Price'].values
df1.drop(['Selling_Price'], axis = 1, inplace = True)
x = df1.copy()

In [ ]:
x.head()

In [ ]:
x['Fuel_Type'] = pd.get_dummies(x['Fuel_Type'])

SPLITTING DATASET

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 2)

In [ ]:
from sklearn.linear_model import LinearRegression
r = LinearRegression()
r.fit(X_train, y_train)

In [ ]:
y_pred = r.predict(X_test)

****PREDICTED VALUE - VS - ACTUAL VALUE****

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
x_grid1 = np.arange(min(y_pred), max(y_pred), .7)
x_grid = np.arange(min(y_test), max(y_test), .7)
x_grid1 = x_grid1.reshape((len(x_grid1), 1))
x_grid = x_grid.reshape((len(x_grid), 1))
plt.scatter(y_pred, y_test, color ='orange')
plt.plot(x_grid)
plt.plot(x_grid1)
plt.xlabel('Predicted value')
plt.ylabel("Test Value");

In [ ]:
plt.plot(y_pred)
plt.plot(y_test)
plt.legend('PT')

***The graph between "PREDICTED VALUE" - VS - "ACTUAL VALUE" is actually very much approximate*** 

> I'll end my notebook here.
Thankyou for going through my notebook.
Have a GOOD DAY!!